In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import concept_table_pipeline as ctp
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [7]:
#define path
data_path = './data/'
output_path = './output/'

In [16]:
with open(data_path+'test1005.out', 'r') as f:
    contents = f.read()

In [10]:
df = pd.read_excel(data_path+'CMS32_DESC_LONG_SHORT_DX.xlsx')

In [11]:
des = list(df.loc[0:79,'LONG DESCRIPTION'])

In [6]:
table = ctp.create_semantic_table(contents,des)

In [13]:
process_index = [index for index in range(len(contents)) if contents.startswith('Processing \'',index)]

In [14]:
len(process_index)

80

In [15]:
len(des)

80

In [12]:
e = ctp.create_semantic_dictionary(contents)
e

{'Disease or Syndrome': ['cholera',
  'typhoid',
  'paratyphoid',
  'paratyphoid b',
  'paratyphoid c',
  'gastroenteritis',
  'septicaemia, nos',
  'infection, nos',
  'meningitis',
  'pneumonia',
  'arthritis',
  'osteomyelitis',
  'shigellosis',
  'dysentery',
  'abscess',
  'amoebiasis',
  'colitis',
  'balantidiosis',
  'giardiasis',
  'coccidiosis',
  'trichomoniasis',
  'cryptosporidiosis',
  'cyclosporiasis',
  'diseases',
  'disease',
  'paracolon infection',
  'enteritis',
  'enterovirus',
  'nec',
  'enteritis due to other viral enteritis'],
 'Functional Concept': ['due', 'due to', 'not'],
 'Bacterium': ['cholerae vibrio',
  'vibrio',
  'salmonella',
  'shigella',
  'staphylococcal',
  'perfringens clostridium',
  'clostridia',
  'parahaemolyticus vibrio',
  'vibrio vulnificus',
  'e coli',
  'bacilli',
  'aerobacter aerogenes',
  'proteus',
  'staphylococcus',
  'pseudomonas',
  'campylobacter',
  'yersinia enterocolitica',
  'difficile clostridium',
  'anaerobes',
  'bacte

In [8]:
table

Disease or Syndrome  \
                                                               cholera   
Cholera due to vibrio cholerae                                       0   
Cholera due to vibrio cholerae el tor                                0   
Cholera, unspecified                                                 0   
Typhoid fever                                                        0   
Paratyphoid fever A                                                  0   
Paratyphoid fever B                                                  0   
Paratyphoid fever C                                                  0   
Paratyphoid fever, unspecified                                       0   
Salmonella gastroenteritis                                           0   
Salmonella septicemia                                                0   
Localized salmonella infection, unspecified                          0   
Salmonella meningitis                                                0   
Salmonella pneumonia                                                 0   
Salmonella arthritis                                                 0   
Salmonella osteomyelitis                                             0   
Other localized salmonella infections                                0   
Other specified salmonella infections                                0   
Salmonella infection, unspecified                                    0   
Shigella dysenteriae                                                 0   
Shigella flexneri                                                    0   
Shigella boydii                                                      0   
Shigella sonnei                                                      0   
Other specified shigella infections                                  0   
Shigellosis, unspecified                                             0   
Staphylococcal food poisoning                                        0   
Botulism food poisoning                                              0   
Food poisoning due to Clostridium perfringens (...                   0   
Food poisoning due to other Clostridia                               0   
Food poisoning due to Vibrio parahaemolyticus                        0   
Food poisoning due to Vibrio vulnificus                              0   
...                                                                ...   
Intestinal infection due to enteropathogenic E....                   0   
Intestinal infection due to enterotoxigenic E. ...                   0   
Intestinal infection due to enteroinvasive E. coli                   0   
Intestinal infection due to enterohemorrhagic E...                   0   
Intestinal infection due to other intestinal E....                   0   
Intestinal infection due to arizona group of pa...                   0   
Intestinal infection due to aerobacter aerogenes                     0   
Intestinal infection due to proteus (mirabilis)...                   0   
Intestinal infection due to staphylococcus                           0   
Intestinal infection due to pseudomonas                              0   
Intestinal infection due to campylobacter                            0   
Intestinal infection due to yersinia enterocoli...                   0   
Intestinal infection due to Clostridium difficile                    0   
Intestinal infection due to other anaerobes                          0   
Intestinal infection due to other gram-negative...                   0   
Intestinal infection due to other organisms                          0   
Bacterial enteritis, unspecified                                     0   
Enteritis due to rotavirus                                           0   
Enteritis due to adenovirus                                          0   
Enteritis due to norwalk virus                                       0   
Enteritis due to other small round viruses [SRV's]                   0   
Enteritis due to calicivirus                           

In [9]:
table.columns.get_level_values(0)

Index(['Disease or Syndrome', 'Disease or Syndrome', 'Disease or Syndrome',
       'Disease or Syndrome', 'Disease or Syndrome', 'Disease or Syndrome',
       'Disease or Syndrome', 'Disease or Syndrome', 'Disease or Syndrome',
       'Disease or Syndrome',
       ...
       'Virus', 'Disease or Syndrome', 'Disease or Syndrome',
       'Disease or Syndrome', 'Organism', 'Functional Concept',
       'Classification', 'Qualitative Concept', 'Classification',
       'Sign or Symptom'],
      dtype='object', length=196)

In [ ]:
table['Disease or Syndrome']

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
categories = list(set(table.columns.get_level_values(0)))
wl = WordNetLemmatizer()
for cat in categories:
    subset = table[cat]
    keywords = list(subset.columns)
    keywords =[keyword.lower() for keyword in keywords]
    wl_keywords=[]
    for keyword in keywords:
        word_pos_tag = nltk.pos_tag([keyword])[0][1]
        wl_keywords.append(wl.lemmatize(keyword, get_wordnet_pos(word_pos_tag)))
    table[cat].columns = wl_keywords
    for new_key in wl_keywords:
        if wl_keywords.count(new_key) > 1:
            array = np.array(wl_keywords)
            index = list(np.where(array==new_key)[0])
            table[(cat,new_key)] = [0 if i==0 else 1 for i in list(subset.iloc[,index].sum(axis=1))]
            

In [ ]:
subset = table['Disease or Syndrome']
wl = WordNetLemmatizer()
keywords = list(subset.columns)
keywords =[keyword.lower() for keyword in keywords]
wl_keywords=[]
for keyword in keywords:
    word_pos_tag = nltk.pos_tag([keyword])[0][1]
    wl_keywords.append(wl.lemmatize(keyword, get_wordnet_pos(word_pos_tag)))

In [ ]:
wl_keywords

In [ ]:
keywords

In [ ]:
keywords

In [ ]:
l = [1,2,0,0,3,1]
s = [0 if i==0 else 1 for i in l]

In [ ]:
s

In [ ]:
subset

In [ ]:
subset = table2['Disease or Syndrome']
subset.columns = ['Apple']*31
table2['Disease or Syndrome']

In [ ]:
len(table2['Disease or Syndrome'].columns)

In [ ]:
table2['Disease or Syndrome'].set_levels(['Apple']*31)

In [23]:
table2 = table.copy()


In [30]:
table2.get_level_values(0)

AttributeError: 'DataFrame' object has no attribute 'get_level_values'

In [34]:
pd.MultiIndex.from_frame(table)

AttributeError: type object 'MultiIndex' has no attribute 'from_frame'

In [36]:
category_table = table.copy()
category_table.columns = category_table.columns.droplevel(1)
category_table.rename(columns = {'category':'Description'}, inplace = True)

In [37]:
category_table

,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,...,Virus,Disease or Syndrome,Disease or Syndrome,Disease or Syndrome,Organism,Functional Concept,Classification,Qualitative Concept,Classification,Sign or Symptom
Cholera due to vibrio cholerae,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cholera due to vibrio cholerae el tor,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Cholera, unspecified",0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Typhoid fever,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paratyphoid fever A,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paratyphoid fever B,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Paratyphoid fever C,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Paratyphoid fever, unspecified",0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Salmonella gastroenteritis,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Salmonella septicemia,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
columns = list(set(category_table.columns[1:]))
df = pd.DataFrame()
for column in columns:
    subset = table[column]
    new_list = list(subset.sum(axis=1))
    new_list =  [0 if i==0 else 1 for i in new_list]
    df[column] = new_list

In [42]:
df.insert(0,'Description',des)

In [43]:
df

,Description,Bacterium,Eukaryote,Disease or Syndrome,Food,Gene or Genome,Idea or Concept,Finding,Temporal Concept,"Body Part, Organ, or Organ Component",...,Body Substance,Intellectual Product,Organism,Plant,Sign or Symptom,Quantitative Concept,Virus,Classification,Spatial Concept,"Amino Acid, Peptide, or Protein,Biologically Active Substance"
0,Cholera due to vibrio cholerae,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cholera due to vibrio cholerae el tor,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Cholera, unspecified",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Typhoid fever,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Paratyphoid fever A,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,Paratyphoid fever B,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,Paratyphoid fever C,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,"Paratyphoid fever, unspecified",0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,Salmonella gastroenteritis,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Salmonella septicemia,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
df.to_csv(output_path+'test_cat.csv',index=False)

In [46]:
df.shape

(80, 26)

In [47]:
df

,Description,Bacterium,Eukaryote,Disease or Syndrome,Food,Gene or Genome,Idea or Concept,Finding,Temporal Concept,"Body Part, Organ, or Organ Component",...,Body Substance,Intellectual Product,Organism,Plant,Sign or Symptom,Quantitative Concept,Virus,Classification,Spatial Concept,"Amino Acid, Peptide, or Protein,Biologically Active Substance"
0,Cholera due to vibrio cholerae,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Cholera due to vibrio cholerae el tor,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Cholera, unspecified",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Typhoid fever,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,Paratyphoid fever A,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,Paratyphoid fever B,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6,Paratyphoid fever C,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,"Paratyphoid fever, unspecified",0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,Salmonella gastroenteritis,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Salmonella septicemia,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
print('apple\'')

apple'


In [ ]:
keywords = 